In [36]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import json
import time
from tqdm.notebook import tqdm 
import numpy as np

<h4>Start cells here if {state}_geo file has not been FIPS encoded

In [2]:
fips_check = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geocoded/geo_sc.csv") #- original geocoded file
fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1","Unnamed: 0.1.1.1","Unnamed: 0.1.1.1.1","Unnamed: 0.1.1.1.1.1", "Unnamed: 0.1.1.1.1.1.1"], axis =1) 
fips_check

,Unnamed: 0,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,0,9547507704,5/1/20,464,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,Sumter,SC,...,325 Chemicals,"3255 Paint, Coating, and Adhesive Manufacturing",3255.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"2410 Highway 15 South, Sumter, SC",33.853682,-80.369867,NaN
1,1,1,9777677704,5/1/20,464,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,North Charleston,SC,...,999 Other,5617 Services to Buildings and Dwellings,5617.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"7684 Southrail Road, North Charleston, SC",32.942332,-80.066165,NaN
2,781333,781333,8416237102,4/15/20,464,PPP,"THOMPSON INDUSTRIAL SERVICES, LLC",104 N MAIN ST,SUMTER,SC,...,999 Other,5629 Remediation and Other Waste Management Se...,5629.0,SOUTH CAROLINA DISTRICT OFFICE,e $5-10 million,500 or more,"104 N MAIN ST, SUMTER, SC",33.921860,-80.341545,4.508500e+14
3,781334,781334,8936977105,4/15/20,464,PPP,HKA ENTERPRISES LLC,337 SPARTANGREEN BLVD,DUNCAN,SC,...,999 Other,"5413 Architectural, Engineering, and Related S...",5413.0,SOUTH CAROLINA DISTRICT OFFICE,e $5-10 million,500 or more,"337 SPARTANGREEN BLVD, DUNCAN, SC",34.918210,-82.095220,4.508302e+14
4,781335,781335,9301987000,4/9/20,464,PPP,HUMAN TECHNOLOGIES INC,105 N Spring St,GREENVILLE,SC,...,999 Other,5613 Employment Services,5613.0,SOUTH CAROLINA DISTRICT OFFICE,e $5-10 million,500 or more,"105 N Spring St, GREENVILLE, SC",34.897892,-82.363490,4.504500e+14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10738,792069,792069,8896048610,3/25/21,464,PPS,ED PERRY AUTO PARTS CO INC.,450 Park St,Inman,SC,...,999 Other,"4413 Automotive Parts, Accessories, and Tire S...",4413.0,SOUTH CAROLINA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"450 Park St, Inman, SC",35.063540,-82.087490,4.508302e+14
10739,792070,792070,9147798304,1/30/21,464,PPS,GET TO IT TRANSPORT INC,126 Vivion Dr,Aiken,SC,...,999 Other,4885 Freight Transportation Arrangement,4885.0,SOUTH CAROLINA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"126 Vivion Dr, Aiken, SC",33.538596,-81.745072,NaN
10740,792071,792071,9543158509,3/12/21,464,PPS,VESPA PIZZA LLC,224 Seven Farms Dr Ste 101,Daniel Island,SC,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,SOUTH CAROLINA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"224 Seven Farms Dr Ste 101, Daniel Island, SC",32.860386,-79.907030,4.501502e+14
10741,792072,792072,9671458706,4/9/21,464,PPS,FRANCHISE CLIQUE LLC,245 Seven Farms Dr Ste 110,Daniel Island,SC,...,999 Other,"5418 Advertising, Public Relations, and Relate...",5418.0,SOUTH CAROLINA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"245 Seven Farms Dr Ste 110, Daniel Island, SC",32.859726,-79.908760,4.501502e+14


In [3]:
columns = ['LoanNumber','full_add', 'Lat', 'Long']
fips_check = fips_check[columns]

In [ ]:
fips_check_nan = fips_check[fips_check["Lat"].isnull()] # check to make sure no nulls in coords
fips_check_nan #3

In [ ]:
fips_check_notna = fips_check[fips_check["Lat"].notna()] # check to make sure no nulls in coords
fips_check_notna.head() #15866

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [3]:
fips_check.to_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/SC_fips_scraped.csv")

In [37]:
fips_check_scraped = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/SC_fips_scraped.csv") 
fips_check_scraped.head() 

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,0,0,0,9547507704,5/1/20,464,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,...,325 Chemicals,"3255 Paint, Coating, and Adhesive Manufacturing",3255.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"2410 Highway 15 South, Sumter, SC",33.853682,-80.369867,4.508500e+14
1,1,1,1,1,9777677704,5/1/20,464,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,...,999 Other,5617 Services to Buildings and Dwellings,5617.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"7684 Southrail Road, North Charleston, SC",32.942332,-80.066165,4.501900e+14
2,2,2,781333,781333,8416237102,4/15/20,464,PPP,"THOMPSON INDUSTRIAL SERVICES, LLC",104 N MAIN ST,...,999 Other,5629 Remediation and Other Waste Management Se...,5629.0,SOUTH CAROLINA DISTRICT OFFICE,e $5-10 million,500 or more,"104 N MAIN ST, SUMTER, SC",33.921860,-80.341545,4.508500e+14
3,3,3,781334,781334,8936977105,4/15/20,464,PPP,HKA ENTERPRISES LLC,337 SPARTANGREEN BLVD,...,999 Other,"5413 Architectural, Engineering, and Related S...",5413.0,SOUTH CAROLINA DISTRICT OFFICE,e $5-10 million,500 or more,"337 SPARTANGREEN BLVD, DUNCAN, SC",34.918210,-82.095220,4.508302e+14
4,4,4,781335,781335,9301987000,4/9/20,464,PPP,HUMAN TECHNOLOGIES INC,105 N Spring St,...,999 Other,5613 Employment Services,5613.0,SOUTH CAROLINA DISTRICT OFFICE,e $5-10 million,500 or more,"105 N Spring St, GREENVILLE, SC",34.897892,-82.363490,4.504500e+14


In [38]:
columns = ['LoanNumber','full_add', 'Lat', 'Long', 'FIPS_z']
fips_check_scraped = fips_check_scraped[columns]

In [39]:
#pd.set_option('display.float_format', '{:.0f}'.format)

In [40]:
fips_check_scraped['Lat'].replace('None', np.nan, inplace=True)
fips_check_scraped['Long'].replace('None', np.nan, inplace=True)

In [41]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna #15866

,LoanNumber,full_add,Lat,Long,FIPS_z
0,9547507704,"2410 Highway 15 South, Sumter, SC",33.853682,-80.369867,4.508500e+14
1,9777677704,"7684 Southrail Road, North Charleston, SC",32.942332,-80.066165,4.501900e+14
2,8416237102,"104 N MAIN ST, SUMTER, SC",33.921860,-80.341545,4.508500e+14
3,8936977105,"337 SPARTANGREEN BLVD, DUNCAN, SC",34.918210,-82.095220,4.508302e+14
4,9301987000,"105 N Spring St, GREENVILLE, SC",34.897892,-82.363490,4.504500e+14
...,...,...,...,...,...
10738,8896048610,"450 Park St, Inman, SC",35.063540,-82.087490,4.508302e+14
10739,9147798304,"126 Vivion Dr, Aiken, SC",33.538596,-81.745072,4.500302e+14
10740,9543158509,"224 Seven Farms Dr Ste 101, Daniel Island, SC",32.860386,-79.907030,4.501502e+14
10741,9671458706,"245 Seven Farms Dr Ste 110, Daniel Island, SC",32.859726,-79.908760,4.501502e+14


In [42]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null.count() #5503

LoanNumber    5
full_add      4
Lat           5
Long          5
FIPS_z        0
dtype: int64

<h4>FIPS Script

In [11]:
#if geocode file
#lat = fips_check_notna["Lat"].astype(float).astype(str).to_list()
#long = fips_check_notna["Long"].astype(float).astype(str).to_list()

#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [12]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
    
service = {'service': Service(ChromeDriverManager().install())}
driver = Browser('chrome', **service, headless=True)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('_x_id').clear()
        driver.find_by_id('_y_id').clear()
        driver.find_by_id('_x_id').fill(long[i])
        driver.find_by_id('_y_id').fill(lat[i])
        driver.find_by_id('proceed').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='pl_gov_census_geo_geocoder_domain_GeographyResult')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('45')] #IN
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                notfound.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [/Users/ciaraspencer/.wdm/drivers/chromedriver/mac64/101.0.4951.41/chromedriver] found in cache


  0%|          | 0/2049 [00:00<?, ?it/s]

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [13]:
geo_id_list =geo_id 

Length Check

In [14]:
print(len(geo_id_list))

2049


Matching FIPS to DataFrame

In [15]:
#for geocode file
#fips_fill = fips_check_notna.iloc[:2684] #confirm pair to notna DataFrame
#if FIPS available
fips_fill = fips_check_scraped_null

In [16]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/t9/c5y08h4d40j0xvl66n01j2680000gn/T/ipykernel_23942/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,full_add,Lat,Long,FIPS_z
0,9547507704,"2410 Highway 15 South, Sumter, SC",33.853682,-80.369867,[450850019032026]
1,9777677704,"7684 Southrail Road, North Charleston, SC",32.942332,-80.066165,[450190031132025]
10,2965127110,"108 Elm St. PO Box 1767, CLEMSON, SC",34.683438,-82.837365,[450770112052011]
15,8511387110,"101 FLINTLAKE RD, COLUMBIA, SC",34.107909,-80.949874,[450790114201014]
22,3206037100,"1141 BOWMAN ROAD STE 101, MOUNT PLEASANT, SC",32.810103,-79.852088,[450190047021003]
...,...,...,...,...,...
10704,1659518309,"941 ALLENDALE FAIRFAX HWY, FAIRFAX, SC",32.961526,-81.241236,[450059705003013]
10716,3621368708,"736 Saint Andrews Rd Ste A, Columbia, SC",34.043335,-81.121590,[450630205102000]
10728,6142438505,"25 Woods Lake Rd Ste 410, Greenville, SC",34.857073,-82.352870,[450450044002001]
10732,6573018303,"930 Pine Hollow Rd, Mount Pleasant, SC",32.809586,-79.868315,[450190046073018]


Remove brackets

In [17]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/t9/c5y08h4d40j0xvl66n01j2680000gn/T/ipykernel_23942/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,full_add,Lat,Long,FIPS_z
0,9547507704,"2410 Highway 15 South, Sumter, SC",33.853682,-80.369867,450850019032026
1,9777677704,"7684 Southrail Road, North Charleston, SC",32.942332,-80.066165,450190031132025
10,2965127110,"108 Elm St. PO Box 1767, CLEMSON, SC",34.683438,-82.837365,450770112052011
15,8511387110,"101 FLINTLAKE RD, COLUMBIA, SC",34.107909,-80.949874,450790114201014
22,3206037100,"1141 BOWMAN ROAD STE 101, MOUNT PLEASANT, SC",32.810103,-79.852088,450190047021003
...,...,...,...,...,...
10704,1659518309,"941 ALLENDALE FAIRFAX HWY, FAIRFAX, SC",32.961526,-81.241236,450059705003013
10716,3621368708,"736 Saint Andrews Rd Ste A, Columbia, SC",34.043335,-81.121590,450630205102000
10728,6142438505,"25 Woods Lake Rd Ste 410, Greenville, SC",34.857073,-82.352870,450450044002001
10732,6573018303,"930 Pine Hollow Rd, Mount Pleasant, SC",32.809586,-79.868315,450190046073018


In [18]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #24

,LoanNumber,full_add,Lat,Long,FIPS_z
773,2269407205,"1921 Falls Valley Drive, RALEIGH, SC",35.898383,-78.607021,NaN
4854,7167417003,"203 E Main St 2nd floor, SPARTANBURG, SC",42.421830,-94.986230,NaN
7483,2754087100,"3545 Highway 17 Bypass, MYRTLE BEACH, SC",28.059534,-81.734188,NaN
9884,4317487702,"8354 State Rd, Philadelphia, SC",40.035567,-75.015204,NaN
10411,9793947702,NaN,36.127071,-79.860718,NaN


Parse for Merge

In [20]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
0,9547507704,450850019032026
1,9777677704,450190031132025
10,2965127110,450770112052011
15,8511387110,450790114201014
22,3206037100,450190047021003
...,...,...
10704,1659518309,450059705003013
10716,3621368708,450630205102000
10728,6142438505,450450044002001
10732,6573018303,450190046073018


Read in base data

In [21]:
#if from geocode file
#fips_check = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geocoded/geo_sc.csv")
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0.1.1.1","Unnamed: 0.1.1.1.1", "Unnamed: 0.1.1.1.1.1", "Unnamed: 0.1.1.1.1.1.1"], axis =1) 
#fips_check

#if from FIPS file
fips_check = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/SC_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0.1.1.1"], axis =1) 
fips_check.head() 


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,0,0,9547507704,5/1/20,464,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,Sumter,...,325 Chemicals,"3255 Paint, Coating, and Adhesive Manufacturing",3255.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"2410 Highway 15 South, Sumter, SC",33.853682,-80.369867,NaN
1,1,1,1,9777677704,5/1/20,464,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,North Charleston,...,999 Other,5617 Services to Buildings and Dwellings,5617.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"7684 Southrail Road, North Charleston, SC",32.942332,-80.066165,NaN
2,2,781333,781333,8416237102,4/15/20,464,PPP,"THOMPSON INDUSTRIAL SERVICES, LLC",104 N MAIN ST,SUMTER,...,999 Other,5629 Remediation and Other Waste Management Se...,5629.0,SOUTH CAROLINA DISTRICT OFFICE,e $5-10 million,500 or more,"104 N MAIN ST, SUMTER, SC",33.921860,-80.341545,4.508500e+14
3,3,781334,781334,8936977105,4/15/20,464,PPP,HKA ENTERPRISES LLC,337 SPARTANGREEN BLVD,DUNCAN,...,999 Other,"5413 Architectural, Engineering, and Related S...",5413.0,SOUTH CAROLINA DISTRICT OFFICE,e $5-10 million,500 or more,"337 SPARTANGREEN BLVD, DUNCAN, SC",34.918210,-82.095220,4.508302e+14
4,4,781335,781335,9301987000,4/9/20,464,PPP,HUMAN TECHNOLOGIES INC,105 N Spring St,GREENVILLE,...,999 Other,5613 Employment Services,5613.0,SOUTH CAROLINA DISTRICT OFFICE,e $5-10 million,500 or more,"105 N Spring St, GREENVILLE, SC",34.897892,-82.363490,4.504500e+14


Merge with MainFrame

In [22]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,0,0,0,9547507704,5/1/20,464,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,Sumter,...,"3255 Paint, Coating, and Adhesive Manufacturing",3255.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"2410 Highway 15 South, Sumter, SC",33.853682,-80.369867,NaN,450850019032026
1,1,1,1,9777677704,5/1/20,464,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,North Charleston,...,5617 Services to Buildings and Dwellings,5617.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"7684 Southrail Road, North Charleston, SC",32.942332,-80.066165,NaN,450190031132025
2,2,781333,781333,8416237102,4/15/20,464,PPP,"THOMPSON INDUSTRIAL SERVICES, LLC",104 N MAIN ST,SUMTER,...,5629 Remediation and Other Waste Management Se...,5629.0,SOUTH CAROLINA DISTRICT OFFICE,e $5-10 million,500 or more,"104 N MAIN ST, SUMTER, SC",33.921860,-80.341545,4.508500e+14,NaN
3,3,781334,781334,8936977105,4/15/20,464,PPP,HKA ENTERPRISES LLC,337 SPARTANGREEN BLVD,DUNCAN,...,"5413 Architectural, Engineering, and Related S...",5413.0,SOUTH CAROLINA DISTRICT OFFICE,e $5-10 million,500 or more,"337 SPARTANGREEN BLVD, DUNCAN, SC",34.918210,-82.095220,4.508302e+14,NaN
4,4,781335,781335,9301987000,4/9/20,464,PPP,HUMAN TECHNOLOGIES INC,105 N Spring St,GREENVILLE,...,5613 Employment Services,5613.0,SOUTH CAROLINA DISTRICT OFFICE,e $5-10 million,500 or more,"105 N Spring St, GREENVILLE, SC",34.897892,-82.363490,4.504500e+14,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10738,10738,792069,792069,8896048610,3/25/21,464,PPS,ED PERRY AUTO PARTS CO INC.,450 Park St,Inman,...,"4413 Automotive Parts, Accessories, and Tire S...",4413.0,SOUTH CAROLINA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"450 Park St, Inman, SC",35.063540,-82.087490,4.508302e+14,NaN
10739,10739,792070,792070,9147798304,1/30/21,464,PPS,GET TO IT TRANSPORT INC,126 Vivion Dr,Aiken,...,4885 Freight Transportation Arrangement,4885.0,SOUTH CAROLINA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"126 Vivion Dr, Aiken, SC",33.538596,-81.745072,NaN,450030212023011
10740,10740,792071,792071,9543158509,3/12/21,464,PPS,VESPA PIZZA LLC,224 Seven Farms Dr Ste 101,Daniel Island,...,7225 Restaurants and Other Eating Places,7225.0,SOUTH CAROLINA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"224 Seven Farms Dr Ste 101, Daniel Island, SC",32.860386,-79.907030,4.501502e+14,NaN
10741,10741,792072,792072,9671458706,4/9/21,464,PPS,FRANCHISE CLIQUE LLC,245 Seven Farms Dr Ste 110,Daniel Island,...,"5418 Advertising, Public Relations, and Relate...",5418.0,SOUTH CAROLINA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"245 Seven Farms Dr Ste 110, Daniel Island, SC",32.859726,-79.908760,4.501502e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [23]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,0,0,0,9547507704,5/1/20,464,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,Sumter,...,3255.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"2410 Highway 15 South, Sumter, SC",33.853682,-80.369867,NaN,450850019032026,450850019032026
1,1,1,1,9777677704,5/1/20,464,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,North Charleston,...,5617.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"7684 Southrail Road, North Charleston, SC",32.942332,-80.066165,NaN,450190031132025,450190031132025
2,2,781333,781333,8416237102,4/15/20,464,PPP,"THOMPSON INDUSTRIAL SERVICES, LLC",104 N MAIN ST,SUMTER,...,5629.0,SOUTH CAROLINA DISTRICT OFFICE,e $5-10 million,500 or more,"104 N MAIN ST, SUMTER, SC",33.921860,-80.341545,4.508500e+14,NaN,450850013002008.0
3,3,781334,781334,8936977105,4/15/20,464,PPP,HKA ENTERPRISES LLC,337 SPARTANGREEN BLVD,DUNCAN,...,5413.0,SOUTH CAROLINA DISTRICT OFFICE,e $5-10 million,500 or more,"337 SPARTANGREEN BLVD, DUNCAN, SC",34.918210,-82.095220,4.508302e+14,NaN,450830231014043.0
4,4,781335,781335,9301987000,4/9/20,464,PPP,HUMAN TECHNOLOGIES INC,105 N Spring St,GREENVILLE,...,5613.0,SOUTH CAROLINA DISTRICT OFFICE,e $5-10 million,500 or more,"105 N Spring St, GREENVILLE, SC",34.897892,-82.363490,4.504500e+14,NaN,450450016003021.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10738,10738,792069,792069,8896048610,3/25/21,464,PPS,ED PERRY AUTO PARTS CO INC.,450 Park St,Inman,...,4413.0,SOUTH CAROLINA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"450 Park St, Inman, SC",35.063540,-82.087490,4.508302e+14,NaN,450830229022004.0
10739,10739,792070,792070,9147798304,1/30/21,464,PPS,GET TO IT TRANSPORT INC,126 Vivion Dr,Aiken,...,4885.0,SOUTH CAROLINA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"126 Vivion Dr, Aiken, SC",33.538596,-81.745072,NaN,450030212023011,450030212023011
10740,10740,792071,792071,9543158509,3/12/21,464,PPS,VESPA PIZZA LLC,224 Seven Farms Dr Ste 101,Daniel Island,...,7225.0,SOUTH CAROLINA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"224 Seven Farms Dr Ste 101, Daniel Island, SC",32.860386,-79.907030,4.501502e+14,NaN,450150204075006.0
10741,10741,792072,792072,9671458706,4/9/21,464,PPS,FRANCHISE CLIQUE LLC,245 Seven Farms Dr Ste 110,Daniel Island,...,5418.0,SOUTH CAROLINA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"245 Seven Farms Dr Ste 110, Daniel Island, SC",32.859726,-79.908760,4.501502e+14,NaN,450150204075032.0


Drop Extra FIPS Columns

In [24]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
#fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,0,0,9547507704,5/1/20,464,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,Sumter,...,325 Chemicals,"3255 Paint, Coating, and Adhesive Manufacturing",3255.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"2410 Highway 15 South, Sumter, SC",33.853682,-80.369867,450850019032026
1,1,1,1,9777677704,5/1/20,464,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,North Charleston,...,999 Other,5617 Services to Buildings and Dwellings,5617.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"7684 Southrail Road, North Charleston, SC",32.942332,-80.066165,450190031132025
2,2,781333,781333,8416237102,4/15/20,464,PPP,"THOMPSON INDUSTRIAL SERVICES, LLC",104 N MAIN ST,SUMTER,...,999 Other,5629 Remediation and Other Waste Management Se...,5629.0,SOUTH CAROLINA DISTRICT OFFICE,e $5-10 million,500 or more,"104 N MAIN ST, SUMTER, SC",33.921860,-80.341545,450850013002008.0
3,3,781334,781334,8936977105,4/15/20,464,PPP,HKA ENTERPRISES LLC,337 SPARTANGREEN BLVD,DUNCAN,...,999 Other,"5413 Architectural, Engineering, and Related S...",5413.0,SOUTH CAROLINA DISTRICT OFFICE,e $5-10 million,500 or more,"337 SPARTANGREEN BLVD, DUNCAN, SC",34.918210,-82.095220,450830231014043.0
4,4,781335,781335,9301987000,4/9/20,464,PPP,HUMAN TECHNOLOGIES INC,105 N Spring St,GREENVILLE,...,999 Other,5613 Employment Services,5613.0,SOUTH CAROLINA DISTRICT OFFICE,e $5-10 million,500 or more,"105 N Spring St, GREENVILLE, SC",34.897892,-82.363490,450450016003021.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10738,10738,792069,792069,8896048610,3/25/21,464,PPS,ED PERRY AUTO PARTS CO INC.,450 Park St,Inman,...,999 Other,"4413 Automotive Parts, Accessories, and Tire S...",4413.0,SOUTH CAROLINA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"450 Park St, Inman, SC",35.063540,-82.087490,450830229022004.0
10739,10739,792070,792070,9147798304,1/30/21,464,PPS,GET TO IT TRANSPORT INC,126 Vivion Dr,Aiken,...,999 Other,4885 Freight Transportation Arrangement,4885.0,SOUTH CAROLINA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"126 Vivion Dr, Aiken, SC",33.538596,-81.745072,450030212023011
10740,10740,792071,792071,9543158509,3/12/21,464,PPS,VESPA PIZZA LLC,224 Seven Farms Dr Ste 101,Daniel Island,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,SOUTH CAROLINA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"224 Seven Farms Dr Ste 101, Daniel Island, SC",32.860386,-79.907030,450150204075006.0
10741,10741,792072,792072,9671458706,4/9/21,464,PPS,FRANCHISE CLIQUE LLC,245 Seven Farms Dr Ste 110,Daniel Island,...,999 Other,"5418 Advertising, Public Relations, and Relate...",5418.0,SOUTH CAROLINA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"245 Seven Farms Dr Ste 110, Daniel Island, SC",32.859726,-79.908760,450150204075032.0


In [25]:
#fips_merge_drop = fips_merge.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop.loc[fips_merge["FIPS_z"].isnull()] #verify #27 , notna #22799 / 22826

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
773,773,782104,782104,2269407205,4/15/20,464,PPP,"RALEIGH PEDIATRIC ASSOCIATES, P.A.",1921 Falls Valley Drive,RALEIGH,...,999 Other,6211 Offices of Physicians,6211.0,SOUTH CAROLINA DISTRICT OFFICE,c $1-2 million,100 to 249,"1921 Falls Valley Drive, RALEIGH, SC",35.898383,-78.607021,NaN
4854,4854,786185,786185,7167417003,4/7/20,464,PPP,UNITED WAY OF THE PIEDMONT,203 E Main St 2nd floor,SPARTANBURG,...,999 Other,8132 Grantmaking and Giving Services,8132.0,SOUTH CAROLINA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"203 E Main St 2nd floor, SPARTANBURG, SC",42.421830,-94.986230,NaN
7483,7483,788814,788814,2754087100,4/11/20,464,PPP,"MURRELLS INLET ASC, LLC",3545 Highway 17 Bypass,MYRTLE BEACH,...,999 Other,6211 Offices of Physicians,6211.0,SOUTH CAROLINA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"3545 Highway 17 Bypass, MYRTLE BEACH, SC",28.059534,-81.734188,NaN
9884,9884,791215,791215,4317487702,5/1/20,464,PPP,"RICHMANS DISTRIBUTION, INC.-SBA SMALL 7A TERM",8354 State Rd,Philadelphia,...,NaN,NaN,NaN,SOUTH CAROLINA DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"8354 State Rd, Philadelphia, SC",40.035567,-75.015204,NaN
10411,10411,791742,791742,9793947702,5/1/20,464,PPP,DANIEL ISLAND VENTURES LLC,NaN,DANIEL ISLAND,...,999 Other,2362 Nonresidential Building Construction,2362.0,SOUTH CAROLINA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,NaN,36.127071,-79.860718,NaN


Convert to File

In [26]:
fips_IN = fips_merge_drop
#fips_TN = fips_merge_drop

In [27]:
fips_IN.to_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/SC_fips_scraped.csv") 

Review & Compare

In [28]:
fips_IN = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/SC_fips_scraped.csv") 
fips_IN

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,0,0,0,9547507704,5/1/20,464,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,...,325 Chemicals,"3255 Paint, Coating, and Adhesive Manufacturing",3255.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"2410 Highway 15 South, Sumter, SC",33.853682,-80.369867,4.508500e+14
1,1,1,1,1,9777677704,5/1/20,464,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,...,999 Other,5617 Services to Buildings and Dwellings,5617.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"7684 Southrail Road, North Charleston, SC",32.942332,-80.066165,4.501900e+14
2,2,2,781333,781333,8416237102,4/15/20,464,PPP,"THOMPSON INDUSTRIAL SERVICES, LLC",104 N MAIN ST,...,999 Other,5629 Remediation and Other Waste Management Se...,5629.0,SOUTH CAROLINA DISTRICT OFFICE,e $5-10 million,500 or more,"104 N MAIN ST, SUMTER, SC",33.921860,-80.341545,4.508500e+14
3,3,3,781334,781334,8936977105,4/15/20,464,PPP,HKA ENTERPRISES LLC,337 SPARTANGREEN BLVD,...,999 Other,"5413 Architectural, Engineering, and Related S...",5413.0,SOUTH CAROLINA DISTRICT OFFICE,e $5-10 million,500 or more,"337 SPARTANGREEN BLVD, DUNCAN, SC",34.918210,-82.095220,4.508302e+14
4,4,4,781335,781335,9301987000,4/9/20,464,PPP,HUMAN TECHNOLOGIES INC,105 N Spring St,...,999 Other,5613 Employment Services,5613.0,SOUTH CAROLINA DISTRICT OFFICE,e $5-10 million,500 or more,"105 N Spring St, GREENVILLE, SC",34.897892,-82.363490,4.504500e+14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10738,10738,10738,792069,792069,8896048610,3/25/21,464,PPS,ED PERRY AUTO PARTS CO INC.,450 Park St,...,999 Other,"4413 Automotive Parts, Accessories, and Tire S...",4413.0,SOUTH CAROLINA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"450 Park St, Inman, SC",35.063540,-82.087490,4.508302e+14
10739,10739,10739,792070,792070,9147798304,1/30/21,464,PPS,GET TO IT TRANSPORT INC,126 Vivion Dr,...,999 Other,4885 Freight Transportation Arrangement,4885.0,SOUTH CAROLINA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"126 Vivion Dr, Aiken, SC",33.538596,-81.745072,4.500302e+14
10740,10740,10740,792071,792071,9543158509,3/12/21,464,PPS,VESPA PIZZA LLC,224 Seven Farms Dr Ste 101,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,SOUTH CAROLINA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"224 Seven Farms Dr Ste 101, Daniel Island, SC",32.860386,-79.907030,4.501502e+14
10741,10741,10741,792072,792072,9671458706,4/9/21,464,PPS,FRANCHISE CLIQUE LLC,245 Seven Farms Dr Ste 110,...,999 Other,"5418 Advertising, Public Relations, and Relate...",5418.0,SOUTH CAROLINA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"245 Seven Farms Dr Ste 110, Daniel Island, SC",32.859726,-79.908760,4.501502e+14
